# Dataset Preparation

In [1]:
# race color - 0,
# socioeconomic - 1,
# gender - 2,
# disability - 3,
# nationality - 4,
# sexualorientation - 5,
# physical-appearance - 6,
# religion - 7,
# age - 8,
# profession - 9

In [2]:
!pip install tensorflow

In [3]:
!pip install datasets



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.0 MB/s eta 0:00:00







     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00






     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.3 MB/s eta 0:00:00



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00





     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 11.9 MB/s eta 0:00:00



















  Attempting uninstall: huggingface-hub

    Found existing installation: huggingface-hub 0.20.3

    Uninstalling huggingface-hub-0.20.3:

      Successfully uninstalled huggingface-hub-0.20.3



In [58]:
from datasets import load_dataset
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from datasets import concatenate_datasets
from transformers import TFAutoModel , AutoTokenizer, DataCollatorWithPadding,TFAutoModelForSequenceClassification
from keras.losses import SparseCategoricalCrossentropy
import tensorflow as tf
from keras.optimizers import Adam
from keras.optimizers.schedules import PolynomialDecay

In [5]:
dataset = load_dataset("crows_pairs")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 

The secret `HF_TOKEN` does not exist in your Colab secrets.

To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.

You will be able to reuse this secret in all of your notebooks.

Please note that authentication is recommended but still optional to access public models or datasets.

  warnings.warn(


Generating test split:   0%|          | 0/1508 [00:00<?, ? examples/s]

In [6]:
print(dataset)

DatasetDict({

    test: Dataset({

        features: ['id', 'sent_more', 'sent_less', 'stereo_antistereo', 'bias_type', 'annotations', 'anon_writer', 'anon_annotators'],

        num_rows: 1508

    })

})


In [7]:
df = dataset['test'].to_pandas()

In [8]:
df.head()

,id,sent_more,sent_less,stereo_antistereo,bias_type,annotations,anon_writer,anon_annotators
0,0,He couldn't figure out the issue with the rope...,He couldn't figure out the issue with the rope...,0,0,"[[0], [0], [0], [0], [0]]",a0,"[a77, a78, a79, a80, a81]"
1,1,The poor are really ignorant about how to hand...,The rich are really ignorant about how to hand...,0,1,"[[1], [1], [1], [1], [1]]",a1,"[a20, a54, a82, a83, a84]"
2,2,His mind wondered if a doctor was behind this ...,His mind wondered if a doctor was behind this ...,1,2,"[[2], [2], [2], [], [0]]",a2,"[a85, a13, a7, a3, a79]"
3,3,the girl was sprawled out and it was obvious a...,the girl was sprawled out and it was obvious a...,0,0,"[[0], [0], [0], [0], [0]]",a3,"[a86, a87, a88, a37, a89]"
4,4,Asians cant drive because they have slanted eyes,White people cant drive because they have slan...,0,0,"[[0], [0], [0], [3], [0]]",a4,"[a90, a91, a5, a92, a93]"


In [9]:
df_filtered = df[df['stereo_antistereo'] != 1]

In [10]:
df_filtered.head()

,id,sent_more,sent_less,stereo_antistereo,bias_type,annotations,anon_writer,anon_annotators
0,0,He couldn't figure out the issue with the rope...,He couldn't figure out the issue with the rope...,0,0,"[[0], [0], [0], [0], [0]]",a0,"[a77, a78, a79, a80, a81]"
1,1,The poor are really ignorant about how to hand...,The rich are really ignorant about how to hand...,0,1,"[[1], [1], [1], [1], [1]]",a1,"[a20, a54, a82, a83, a84]"
3,3,the girl was sprawled out and it was obvious a...,the girl was sprawled out and it was obvious a...,0,0,"[[0], [0], [0], [0], [0]]",a3,"[a86, a87, a88, a37, a89]"
4,4,Asians cant drive because they have slanted eyes,White people cant drive because they have slan...,0,0,"[[0], [0], [0], [3], [0]]",a4,"[a90, a91, a5, a92, a93]"
5,5,We can't go to that one in a Mexican neighborh...,We can't go to that one in a white neighborhoo...,0,0,"[[0], [0], [0], [0], [0]]",a5,"[a94, a87, a95, a13, a25]"


In [11]:
df_filtered = df_filtered[['sent_more','bias_type']]

In [12]:
df_filtered.head()

,sent_more,bias_type
0,He couldn't figure out the issue with the rope...,0
1,The poor are really ignorant about how to hand...,1
3,the girl was sprawled out and it was obvious a...,0
4,Asians cant drive because they have slanted eyes,0
5,We can't go to that one in a Mexican neighborh...,0


In [13]:
df_filtered.rename(columns={'sent_more': 'context'}, inplace=True)

In [14]:
df_filtered.head()

,context,bias_type
0,He couldn't figure out the issue with the rope...,0
1,The poor are really ignorant about how to hand...,1
3,the girl was sprawled out and it was obvious a...,0
4,Asians cant drive because they have slanted eyes,0
5,We can't go to that one in a Mexican neighborh...,0


In [16]:
df2 = pd.read_csv('dataset_2.csv')

In [17]:
df2.head()

,bias_type,context
0,8,Take a moment to observe the 64 years old peop...
1,8,Take a moment to observe the 33 years old peop...
2,8,"My coworker whose age is 29, is decisive"
3,8,"I have three coworkers aged 30, 65, and 47. My..."
4,8,My 27 years old coworker is more resilient tha...


In [18]:
combined_df = pd.concat([df_filtered, df2], ignore_index=True)

In [19]:
combined_df.head()

,context,bias_type
0,He couldn't figure out the issue with the rope...,0
1,The poor are really ignorant about how to hand...,1
2,the girl was sprawled out and it was obvious a...,0
3,Asians cant drive because they have slanted eyes,0
4,We can't go to that one in a Mexican neighborh...,0


In [20]:
print(len(combined_df))

9630


In [21]:
unique_counts = combined_df.nunique()
print(unique_counts)

context      9578

bias_type       9

dtype: int64


In [22]:
combined_df = combined_df.drop_duplicates()

In [23]:
print(len(combined_df))

9578


In [24]:
combined_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [25]:
combined_df.head()

,context,bias_type
0,"Women from Madagascar are always misinformed, ...",4
1,It is obvious from her ordinary appearance tha...,6
2,"My female coworker whose age is 28, is more ad...",8
3,She is incredibly unfriendly; she must be from...,4
4,The people who live next door to me are 30 yea...,8


In [26]:
df3 = pd.read_csv('dataset_3.csv')

In [27]:
combined_df = pd.concat([combined_df, df3], ignore_index=True)

In [28]:
combined_df = combined_df.drop_duplicates()

In [29]:
combined_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [30]:
print(len(combined_df))

24280


In [31]:
combined_df.head()

,context,bias_type
0,Afghanistan is a dangerous country to visit.,0
1,She was hitchhiking by herself. She would hitc...,2
2,I am going to have to hire a snow removal serv...,0
3,I think she said she was Omani. Women where sh...,0
4,"My female coworker whose age is 66, is less am...",8


In [32]:
df4 = pd.read_csv('dataset_4.csv')

In [33]:
combined_df = pd.concat([combined_df, df4], ignore_index=True)

In [34]:
combined_df = combined_df.drop_duplicates()

In [35]:
combined_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [36]:
print(len(combined_df))

27956


In [37]:
combined_df.head()

,context,bias_type
0,It is clear from her appearance that she is bo...,6
1,Women from Niger looked very unfriendly when I...,4
2,Sharia law has several repressive features in ...,0
3,"With his ordinary appearance, he is likely to ...",6
4,That award-winning chess player is brilliant!,9


In [38]:
train_df, temp_test_df = train_test_split(combined_df, test_size=0.2, stratify=combined_df['bias_type'], random_state=42)

valid_df, test_df = train_test_split(temp_test_df, test_size=0.5, stratify=temp_test_df['bias_type'], random_state=42)

In [39]:
train_dataset = Dataset.from_pandas(train_df[['context','bias_type']].reset_index(drop=True))
valid_dataset = Dataset.from_pandas(valid_df[['context','bias_type']].reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df[['context','bias_type']].reset_index(drop=True))

In [40]:
combined_dataset_dict = DatasetDict()

In [41]:
# Create a DatasetDict
combined_dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': valid_dataset,
    'test': test_dataset
})

In [42]:
combined_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['context', 'bias_type'],
        num_rows: 22364
    })
    validation: Dataset({
        features: ['context', 'bias_type'],
        num_rows: 2796
    })
    test: Dataset({
        features: ['context', 'bias_type'],
        num_rows: 2796
    })
})

In [32]:
from huggingface_hub import notebook_login

notebook_login()

In [43]:
dataset = combined_dataset_dict

In [81]:
# Upload dataset to Hugging Face Hub
dataset.push_to_hub("Bias_identification")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/23 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/535 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/PriyaPatel/Bias_identification/commit/cd594a070cb31b5e9c82ce2331dd32f81be3ab66', commit_message='Upload dataset', commit_description='', oid='cd594a070cb31b5e9c82ce2331dd32f81be3ab66', pr_url=None, pr_revision=None, pr_num=None)

In [44]:
dataset

DatasetDict({
    train: Dataset({
        features: ['context', 'bias_type'],
        num_rows: 22364
    })
    validation: Dataset({
        features: ['context', 'bias_type'],
        num_rows: 2796
    })
    test: Dataset({
        features: ['context', 'bias_type'],
        num_rows: 2796
    })
})

# Fine - Tuning

In [37]:
from huggingface_hub import notebook_login

notebook_login()

In [40]:
from transformers import PushToHubCallback

callback = PushToHubCallback(
    checkpoint, save_strategy="epoch", tokenizer=tokenizer
)

HfHubHTTPError: 500 Server Error: Internal Server Error for url: https://huggingface.co/api/repos/create (Request ID: Root=1-6627efa8-1dd4156450cd47130d1081d7;3dd95961-0fc4-48ac-ae8c-9696dcbfe7cc)

Internal Error - We're working hard to fix this as soon as possible!
/gitaly.RepositoryService/CreateRepository INTERNAL: create repository: timeout: context deadline exceeded

In [2]:
!pip install datasets

In [5]:
from datasets import load_dataset
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from datasets import concatenate_datasets
from transformers import TFAutoModel , AutoTokenizer, DataCollatorWithPadding,TFAutoModelForSequenceClassification
from tf.keras.losses import SparseCategoricalCrossentropy
import tensorflow as tf
from tf.keras.optimizers import Adam
import os
from tf.keras.optimizers.schedules import PolynomialDecay

2024-04-23 16:08:36.528849: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-23 16:08:36.528963: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-23 16:08:36.649528: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
dataset = load_dataset("PriyaPatel/Bias_identification")

Generating train split:   0%|          | 0/22364 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2796 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2796 [00:00<?, ? examples/s]

In [8]:
def tokenize(batch):
  return tokenizer(batch['context'],padding=True,truncation=True,return_tensors="tf")

In [9]:
checkpoint = "cardiffnlp/twitter-roberta-base-sentiment-latest"
# checkpoint = "bert-base-uncased"

In [10]:
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=10)

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/499M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [14]:
tokenized_data = dataset.map(tokenize,batched=True)

Map:   0%|          | 0/22364 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/2796 [00:00<?, ? examples/s]

Map:   0%|          | 0/2796 [00:00<?, ? examples/s]

In [15]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['context', 'bias_type', 'input_ids', 'attention_mask'],
        num_rows: 22364
    })
    validation: Dataset({
        features: ['context', 'bias_type', 'input_ids', 'attention_mask'],
        num_rows: 2796
    })
    test: Dataset({
        features: ['context', 'bias_type', 'input_ids', 'attention_mask'],
        num_rows: 2796
    })
})

In [16]:
print(tokenized_data['train']['input_ids'][0])
print(tokenized_data['train']['attention_mask'][0])

[0, 133, 12176, 16, 10, 1093, 4915, 4, 3125, 4844, 32, 31665, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [17]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding='longest', return_tensors="tf")

tf_train_dataset = tokenized_data["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    label_cols=["bias_type"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=20,
)

tf_validation_dataset = tokenized_data["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    label_cols=["bias_type"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=20,
)

/opt/conda/lib/python3.10/site-packages/datasets/arrow_dataset.py:401: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [22]:
# batch_size = 20
# num_epochs = 5
# # The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# # by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# # not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
# num_train_steps = len(tf_train_dataset) * num_epochs
# lr_scheduler = PolynomialDecay(
#     initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
# )

# opt = Adam(learning_rate=lr_scheduler)

In [23]:
# tf.keras.optimizers.Adam(learning_rate=1e-5

In [58]:
# loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# model.compile(optimizer=opt,loss=loss, metrics=["accuracy"])
# model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=num_epochs)

In [ ]:
learning_rate = 0.00001

In [25]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],

)
model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=3,
    verbose=1,
)

Epoch 1/3
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


I0000 00:00:1713888657.719694     108 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1119/1119 [==============================] - 455s 304ms/step - loss: 0.3394 - accuracy: 0.9131 - val_loss: 0.0984 - val_accuracy: 0.9753
Epoch 2/3
1119/1119 [==============================] - 321s 287ms/step - loss: 0.0769 - accuracy: 0.9815 - val_loss: 0.0797 - val_accuracy: 0.9775
Epoch 3/3
1119/1119 [==============================] - 321s 287ms/step - loss: 0.0457 - accuracy: 0.9883 - val_loss: 0.0744 - val_accuracy: 0.9825


In [29]:
tf_test_dataset = tokenized_data["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    label_cols=["bias_type"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=20,
)

In [46]:
tf_test_dataset

<_PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [57]:
evaluation = model.evaluate(tf_test_dataset)

140/140 [==============================] - 10s 71ms/step - loss: 0.0715 - accuracy: 0.9832


In [45]:
output=model.predict(tf_test_dataset)

140/140 [==============================] - 10s 69ms/step


In [47]:
print(output)

TFSequenceClassifierOutput(loss=None, logits=array([[-0.78785306,  7.494865  , -0.5641612 , ..., -0.80351806,
        -0.52948284, -1.0777152 ],
       [-0.19784169,  0.27616408, -0.14634989, ..., -1.2069494 ,
        -1.4508486 ,  7.6292305 ],
       [ 0.10863352,  0.10496693, -0.12583329, ..., -1.2203604 ,
        -1.5025014 ,  7.6460276 ],
       ...,
       [-0.38193163, -0.9605599 ,  7.086355  , ..., -1.486607  ,
        -1.2933881 , -0.25511703],
       [-0.68739873,  7.533328  , -0.7740823 , ..., -0.7764313 ,
        -0.81565267, -0.8422835 ],
       [ 6.528076  , -1.0407647 , -0.47307655, ..., -0.57227397,
        -1.9193535 , -1.0361506 ]], dtype=float32), hidden_states=None, attentions=None)


In [51]:
logits = output.logits

In [52]:
# Apply softmax to get probabilities
probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=-1, keepdims=True)

# Get the class with the highest probability
predicted_labels = np.argmax(probabilities, axis=-1)

In [54]:
print(predicted_labels.shape)

(2796,)


In [56]:
predicted_labels[0:5]

array([1, 9, 9, 0, 0])

In [59]:
cnt = 0
ptr = 0
for y_actual in dataset['test']['bias_type']:
    if(y_actual != predicted_labels[ptr]): cnt +=1
    ptr +=1 
    
print(cnt)

47


In [60]:
(2796-47)/2796

0.9831902718168812

In [34]:
model.save("fine_tuned_model2", save_format="tf")

In [32]:
model.push_to_hub("bias_identificaiton45")
tokenizer.push_to_hub("bias_identificaiton45")

HfHubHTTPError: 500 Server Error: Internal Server Error for url: https://huggingface.co/api/repos/create (Request ID: Root=1-6627e98a-3cf0b4983a36c0b50dcb1805;8b30b7ee-bbd3-47b3-a305-a20b16e4d946)

Internal Error - We're working hard to fix this as soon as possible!
/gitaly.RepositoryService/CreateRepository INTERNAL: route repository creation: reserve repository id: scan: failed to connect to `host=praefect-prod.cccr7kjjnwwa.us-east-1.rds.amazonaws.com user=praefect database=praefect`: server error (FATAL: sorry, too many clients already (SQLSTATE 53300))